In [ ]:
!pip install torch torchvision torchaudio
!pip install peft
!pip install transformers
!pip install urlextract
!pip install pandas numpy

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

tokenizer = AutoTokenizer.from_pretrained("VRLLab/TurkishBERTweet")
turkishBERTweet = AutoModel.from_pretrained("VRLLab/TurkishBERTweet")

turkishBERTweet.to(device)

cuda


Some weights of RobertaModel were not initialized from the model checkpoint at VRLLab/TurkishBERTweet and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(100000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (d

In [158]:
def create_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=256).to(device)

    with torch.no_grad():
        outputs = turkishBERTweet(**inputs)

    last_hidden_states = outputs.last_hidden_state
    cls_embedding = last_hidden_states[:, 0, :]

    return cls_embedding.cpu().numpy().flatten().tolist()

In [160]:
import pandas

df_ub = pd.read_csv('ub_2em_clean.csv', index_col=0)
df_ub['embeds'] = df_ub['biography'].progress_apply(create_embeddings)
df_ub.to_csv('ub_em.csv')
df_ub

  0%|          | 0/3253 [00:00<?, ?it/s]

,username,biography,embeds
0,deparmedya,<hashtag> mediaplanning </hashtag> <hashtag> m...,"[-0.14732463657855988, 0.012161445803940296, -..."
2,kafesfirin,<emoji> yuvarlak_raptiye </emoji> söğütözü <em...,"[-0.18014419078826904, 0.0010251292260363698, ..."
3,vimerang,dijital i̇letişim yönetimi <emoji> klaket_taht...,"[-0.13595470786094666, 0.004898972809314728, -..."
10,mustafa_yalcinn38,talas belediye başkanı,"[-0.12611980736255646, -0.003150859847664833, ..."
12,drtubagunebak,<emoji> kadın_sağlık_çalışanı_açık_ten_tonu </...,"[-0.18248912692070007, -0.014781055971980095, ..."
...,...,...,...
5407,bs_forklift,kuruluş tarihi : 01.10.2014,"[-0.13339956104755402, 0.012177901342511177, -..."
5408,woodtechistanbul,avrasya'nın lider ağaç i̇şleme teknolojileri f...,"[-0.16302429139614105, 0.017178848385810852, 0..."
5409,netsvadi,<emoji> karla_kaplı_dağ </emoji> bornova kayad...,"[-0.16384869813919067, -0.010935492813587189, ..."
5410,herbisiatolye,<emoji> fide </emoji> şifalı bitkiler atölyesi...,"[-0.17341944575309753, -0.00872002448886633, -..."


In [162]:
import pandas

df_uc = pd.read_csv('uc_2em_clean.csv', index_col=0)
df_uc["embeds"] = df_uc["caption"].progress_apply(create_embeddings)

df_uc.to_csv('uc_em.csv')
df_uc

  0%|          | 0/111117 [00:00<?, ?it/s]

,username,caption,embed
0,deparmedya,cumhuriyetimizin 100.yılı kutlu olsun <emoji> ...,"[-0.1364944577217102, 0.016088083386421204, 0...."
1,deparmedya,oriflame duologi lansmanı <hashtag> isveçtenge...,"[-0.14490976929664612, 0.013284746557474136, -..."
2,deparmedya,<hashtag> oriflameilesaçbakımdevrimi </hashtag...,"[-0.13719674944877625, 0.011155212298035622, -..."
3,deparmedya,<emoji> sıkıştıran_el_koyu_ten_tonu </emoji> <...,"[-0.15708720684051514, 0.015240379609167576, -..."
4,deparmedya,07 agustos’23 <hashtag> oriflameturkiye </hash...,"[-0.1448381245136261, 0.019869213923811913, 0...."
...,...,...,...
172998,tariszeytinyagi,"9 eylül güzel i̇zmir’imizin kurtuluş günü, cum...","[-0.08002351969480515, 0.04977072402834892, 0...."
172999,tariszeytinyagi,tariş zeytinyağı sofralarımıza uzanan şifa ve ...,"[-0.11279517412185669, 0.022404693067073822, 0..."
173000,tariszeytinyagi,kendine özgün tadı ile damağınızda derin bir l...,"[-0.0806884840130806, 0.05599124729633331, 0.0..."
173001,tariszeytinyagi,"serinleten ayran çorbası malzemeler: 1,5 su ba...","[-0.07036983966827393, 0.08091183006763458, 0...."
